# Assignment *3* - CS598 - Deep Learning 

__MNIST with Tensorflow/CNN__

## Libraries 

For this homework, it is using tensorflow for the MNIST problem with CNN approach. Based on the example in the textbook, libraries being used are mainly from tensorflow, including mnist dataset.

*Import warnings library to ignore any warnings*

In [1]:
import warnings
warnings.filterwarnings('ignore')

*Check the version of tensorflow to verify it will work with the examople from the textbook (1.14.0)*

In [2]:
import tensorflow as tf
print(tf.__version__)

1.14.0


## Dataset loading

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
batchSz = 100
learningRate = 0.5
epoches = 10 

# calculate number of batches for training/test data
trainingBatches = int(len(mnist.train.labels) / batchSz) * epoches 
testBatches = int(len(mnist.test.labels) / batchSz)

In [5]:
img = tf.placeholder(tf.float32, [batchSz, 784])
ans = tf.placeholder(tf.float32, [batchSz, 10])

##### Simple TF Convolution Example from textbook

In [6]:
"""
W = tf.Variable(tf.random_normal([784, 10],stddev=.1))
b = tf.Variable(tf.random_normal([10],stddev=.1))

#reshape the input images
image = tf.reshape(img, [batchSz, 28, 28, 1]) 
# initialize filter parameters, size, channel, number of features 
flts = tf.Variable(tf.truncated_normal([5, 5, 1, 4], stddev = 0.1))
# convolution layer parameters, input, strike, channel 
convOut = tf.nn.conv2d(image, flts, [1, 2, 2, 1], "SAME")
# activation 
convOut = tf.nn.relu(convOut)
# flatten the output 784 = 14(28/2) * 14(28/2) * 4
convOut = tf.reshape(convOut, [100, 784])
# apply the linear function
prbs = tf.nn.softmax(tf.matmul(convOut, W) + b)
"""
pass

##### Multilevel Convolution implementation 

In [7]:
#reshape the input images
image = tf.reshape(img, [batchSz, 28, 28, 1])

##### First level #####
# filter initialization, size, channel, number of features
flts = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1))
# convolution layer parameters, input, strike, channel 
convOut = tf.nn.conv2d(image, flts, [1, 2, 2, 1], "SAME")
# activation 
convOut = tf.nn.relu(convOut)

##### Second level #####
# filter initialization, size, channel(32 is the features from first level)
# number of features 
flts2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
# convolution layer parameters, input, strike, channel 
convOut2 = tf.nn.conv2d(convOut, flts2, [1,  2,  2,  1], "SAME")

# flatten the output 3136 = 7(28/2/2) * 7(28/2/2) * 64(features from flts2) 
convOut2 = tf.reshape(convOut2, [batchSz, 3136])

# weights/biases 
W = tf.Variable(tf.random_normal([3136,10],stddev=0.1))
b = tf.Variable(tf.random_normal([10], stddev=0.1))

# apply the linear function to get probability distributions 
prbs = tf.nn.softmax(tf.matmul(convOut2, W) + b)

In [8]:
xEnt = tf.reduce_mean(-tf.reduce_sum(ans * tf.log(prbs), reduction_indices=[1]))

In [9]:
train = tf.train.GradientDescentOptimizer(learningRate).minimize(xEnt)

In [10]:
numCorrect = tf.equal(tf.argmax(prbs,1), tf.argmax(ans,1))

In [11]:
accuracy = tf.reduce_mean(tf.cast(numCorrect, tf.float32))

In [12]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [13]:
for i in range(trainingBatches):
    imgs, anss = mnist.train.next_batch(batchSz)
    # textbook p.38, python's combining computation
    train_acc,ignore = sess.run([accuracy,train], feed_dict = {img: imgs, ans: anss})
    if i % 100 == 0:
        print("Batch: %r, Train Accuracy: %r" % (i, train_acc))

Batch: 0, Train Accuracy: 0.08
Batch: 100, Train Accuracy: 0.93
Batch: 200, Train Accuracy: 0.95
Batch: 300, Train Accuracy: 0.99
Batch: 400, Train Accuracy: 0.93
Batch: 500, Train Accuracy: 0.95
Batch: 600, Train Accuracy: 0.99
Batch: 700, Train Accuracy: 0.97
Batch: 800, Train Accuracy: 0.99
Batch: 900, Train Accuracy: 0.95
Batch: 1000, Train Accuracy: 1.0
Batch: 1100, Train Accuracy: 0.96
Batch: 1200, Train Accuracy: 0.98
Batch: 1300, Train Accuracy: 0.99
Batch: 1400, Train Accuracy: 0.96
Batch: 1500, Train Accuracy: 0.95
Batch: 1600, Train Accuracy: 0.96
Batch: 1700, Train Accuracy: 0.95
Batch: 1800, Train Accuracy: 0.99
Batch: 1900, Train Accuracy: 0.99
Batch: 2000, Train Accuracy: 0.99
Batch: 2100, Train Accuracy: 0.99
Batch: 2200, Train Accuracy: 0.99
Batch: 2300, Train Accuracy: 0.99
Batch: 2400, Train Accuracy: 0.98
Batch: 2500, Train Accuracy: 0.97
Batch: 2600, Train Accuracy: 0.98
Batch: 2700, Train Accuracy: 0.99
Batch: 2800, Train Accuracy: 1.0
Batch: 2900, Train Accuracy:

In [14]:
sumAcc=0
for i in range(testBatches):
    imgs, anss = mnist.test.next_batch(batchSz)
    sumAcc+=sess.run(accuracy, feed_dict={img: imgs, ans: anss})
print("Test Accuracy: %r" % (sumAcc/testBatches))

Test Accuracy: 0.9769000095129013


In [15]:
# save the trained model 
model_saver = tf.train.Saver()
save_path = model_saver.save(sess, 'Model/tf_mnist_cnn.ckpt')
print('Model has been saved to path:', save_path)

Model has been saved to path: Model/tf_mnist_cnn.ckpt


## References

*Introduction to Deep Learning by Eugene Charniak*